In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

問題50

In [2]:
# データ読み込んで覗いてみる
data = pd.read_csv("../data/news+aggregator/newsCorpora.csv", sep="\t", header=None)
data.columns = ["ID", "Title", "URL", "Publisher", "Category", "Story", "Hostname", "Timestamp"]
data.head(10)

,ID,Title,URL,Publisher,Category,Story,Hostname,Timestamp
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
5,6,Plosser: Fed May Have to Accelerate Tapering Pace,http://www.nasdaq.com/article/plosser-fed-may-...,NASDAQ,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.nasdaq.com,1394470372212
6,7,Fed's Plosser: Taper pace may be too slow,http://www.marketwatch.com/story/feds-plosser-...,MarketWatch,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.marketwatch.com,1394470372405
7,8,Fed's Plosser expects US unemployment to fall ...,http://www.fxstreet.com/news/forex-news/articl...,FXstreet.com,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.fxstreet.com,1394470372615
8,9,US jobs growth last month hit by weather:Fed P...,http://economictimes.indiatimes.com/news/inter...,Economic Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,economictimes.indiatimes.com,1394470372792
9,10,ECB unlikely to end sterilisation of SMP purch...,http://www.iii.co.uk/news-opinion/reuters/news...,Interactive Investor,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.iii.co.uk,1394470501265


In [3]:
# カテゴリの均衡チェック
data["Category"].value_counts()

Category
e    152469
b    115967
t    108344
m     45639
Name: count, dtype: int64

In [4]:
# train 80%, test 20% に分けてから、 test 20%をさらに1:1に分けた
xtrain, xtest, ytrain, ytest = train_test_split(data.drop("Category", axis=1), data["Category"], train_size=0.8, test_size=0.2, random_state=666)
temp = pd.concat([xtest, ytest], axis=1)
xval, xtest, yval, ytest = train_test_split(temp.drop("Category", axis=1), temp["Category"], train_size=0.5, test_size=0.5, random_state=777)

In [5]:
# 後の問題のために出力
out_train = pd.concat([xtrain["Title"], ytrain], axis=1)
out_val = pd.concat([xval["Title"], yval], axis=1)
out_test = pd.concat([xtest["Title"], ytest], axis=1)
out_train.to_csv("../data/train.txt", sep="\t")
out_val.to_csv("../data/val.txt", sep="\t")
out_test.to_csv("../data/test.txt", sep="\t")

問題51

In [13]:
# 前処理のためやり直す
data = pd.read_csv("../data/news+aggregator/newsCorpora.csv", sep="\t", header=None)
data.columns = ["ID", "Title", "URL", "Publisher", "Category", "Story", "Hostname", "Timestamp"]

#前処理
# ID, URL, Story, Timestampはたぶん絶対いらない
data = data.drop(["ID", "URL", "Story", "Timestamp"], axis=1)

# データの分割やり直し
x_train, x_test, y_train, y_test = train_test_split(data.drop("Category", axis=1), data["Category"], train_size=0.8, test_size=0.2, random_state=666)
temp = pd.concat([xtest, ytest], axis=1)
x_val, x_test, y_val, y_test = train_test_split(temp.drop("Category", axis=1), temp["Category"], train_size=0.5, test_size=0.5, random_state=777)

In [12]:
# 特徴量はSentenceBERTで計算したEmbeddingsを用いる
model = SentenceTransformer("all-mpnet-base-v2") # see https://huggingface.co/sentence-transformers/all-mpnet-base-v2

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
x_train["Title"] = x_train["Title"].apply(model.encode)

TypeError: 'float' object is not subscriptable

In [ ]:
x_train["Publisher"] = x_train["Publisher"].map(lambda x: model.encode(str(x)))
x_train["Hostname"] = x_train["Hostname"].apply(model.encode)

In [ ]:
x_val["Title"] = x_val["Title"].map(lambda x: model.encode(str(x)))
x_val["Publisher"] = x_val["Publisher"].map(lambda x: model.encode(str(x)))
x_val["Hostname"] = x_val["Hostname"].map(lambda x: model.encode(str(x)))
x_test["Title"] = x_val["Title"].map(lambda x: model.encode(str(x)))
x_test["Publisher"] = x_val["Publisher"].map(lambda x: model.encode(str(x)))
x_test["Hostname"] = x_val["Hostname"].map(lambda x: model.encode(str(x)))